# 04. Merging Datasets

Merging is your new favourite word. Merging is one of the most powerful skills in data analysis, and it is what separates the little girls – with their Excel tables – from big powerful boss women like you.

It's hard to convey this in words, so let us get right to examples. 

Let's load the IDS table and the clean stroke count table that Marina so kindly cleaned for us in the last notebook:

In [26]:
import pandas as pd

# Load the IDS table
ids_df = pd.read_csv('../daniel_tables/ids_df.csv',
    index_col=None,
    encoding='utf-8')

# cleaning
ids_df = ids_df.drop_duplicates()
ids_df = ids_df.dropna(subset=['character', 'components'], how='any')
ids_df = ids_df[~ids_df['components'].str.contains('←')]
ids_df = ids_df[~ids_df['components'].str.contains('→')]
ids_df = ids_df[~ids_df['components'].str.contains('CDP')].copy()

# Load the clean stroke count table
stroke_df = pd.read_csv('../marina_tables/stroke_count_clean.csv',
    index_col=None,
    encoding='utf-8')

# Print the two tables' shapes
print(f"\nIDS Table shape: {ids_df.shape}")
print(f"\nStroke Count Table shape: {stroke_df.shape}")



IDS Table shape: (161368, 3)

Stroke Count Table shape: (89586, 3)


We have two tables. The IDS table has character decomposition information, and the stroke count table has stroke counts. Wouldn't it be nice to have all of that on one table?

That's what merging is for! But here's the thing: not every character in the IDS table has a stroke count, and not every character in the stroke count table has IDS decomposition data. So we need to decide: what do we want to keep?

Pandas gives us several options, and they're called "joins". Let's see what happens with each one:

## Inner Join: Only Matching Characters

An inner join keeps ONLY the rows where the character exists in BOTH tables. This is the most restrictive option - if a character doesn't have a stroke count, it gets dropped. If a character doesn't have IDS data, it gets dropped.

Use this when you only want complete data - characters that have information in both tables.


In [27]:
# Inner join: only keep characters that exist in BOTH tables
df_inner = pd.merge(ids_df, stroke_df,
                    on='character',
                    how='inner')

print(f"Original IDS table: {len(ids_df)} rows")
print(f"Original stroke count table: {len(stroke_df)} rows")
print(f"After inner join: {len(df_inner)} rows")
print(f"\nLost {len(ids_df) - len(df_inner)} rows from IDS table")
print("\nFirst few rows:")
print(df_inner.head())


Original IDS table: 161368 rows
Original stroke count table: 89586 rows
After inner join: 151660 rows

Lost 9708 rows from IDS table

First few rows:
  code_point_x character components code_point_y stroke_count
0      U+04E00         一          一       U+4E00            1
1      U+0620C         戌        ⿵戊一       U+620C            6
2      U+06B72         歲        ⿻步戌       U+6B72           13
3      U+0528C         劌        ⿰歲刂       U+528C           15
4      U+05666         噦        ⿰口歲       U+5666           16


## Left Join: Keep All IDS Characters

A left join keeps ALL rows from the LEFT table (ids_df) and adds matching data from the RIGHT table (stroke_df) where it exists. If there's no match, the stroke count columns will be empty (NaN).

Use this when you want to keep all your IDS data, even if some characters don't have stroke counts yet.

**Note:** When each table has an identically-named column, if you don't merge on that column, you get _x and _y added to the end. In this case, we can't merge on the code_point column, because the one has an extra 0 in the codes. Instead, we will delete this from one of the tables.


In [28]:
# Delete code_point column from stroke_df
stroke_df = stroke_df.drop(columns=['code_point'])

# Left join: keep all IDS characters, add stroke counts where available
df_left = pd.merge(ids_df, stroke_df,
                   on='character',
                   how='left')

print(f"Original IDS table: {len(ids_df)} rows")
print(f"After left join: {len(df_left)} rows")
print(f"\nKept all {len(ids_df)} rows from IDS table!")
print(f"Added stroke counts for {df_left['stroke_count'].notna().sum()} characters")
print(f"Missing stroke counts for {df_left['stroke_count'].isna().sum()} characters")
print("\nFirst few rows:")
print(df_left.head())
print("\nSome rows with missing stroke counts:")
print(df_left[df_left['stroke_count'].isna()].head())


Original IDS table: 161368 rows
After left join: 161368 rows

Kept all 161368 rows from IDS table!
Added stroke counts for 151660 characters
Missing stroke counts for 9708 characters

First few rows:
  code_point character components stroke_count
0    U+04E00         一          一            1
1    U+0620C         戌        ⿵戊一            6
2    U+06B72         歲        ⿻步戌           13
3    U+0528C         劌        ⿰歲刂           15
4    U+05666         噦        ⿰口歲           16

Some rows with missing stroke counts:
      code_point character components stroke_count
32884     U+9FEB         鿫     ⿹气奥[G]          NaN
32885     U+9FEC         鿬        ⿰石田          NaN
32886     U+9FED         鿭        ⿰钅尔          NaN
32887     U+9FEE         鿮        ⿸广帯          NaN
32888     U+9FEF         鿯      ⿱物⿱人米          NaN


## Right Join: Keep All Stroke Count Characters

A right join keeps ALL rows from the RIGHT table (stroke_df) and adds matching data from the LEFT table (ids_df) where it exists. If there's no match, the IDS columns will be empty (NaN).

Use this when you want to keep all your stroke count data, even if some characters don't have IDS decomposition yet.


In [29]:
# Right join: keep all stroke count characters, add IDS data where available
df_right = pd.merge(ids_df, stroke_df,
                    on='character',
                    how='right')

print(f"Original stroke count table: {len(stroke_df)} rows")
print(f"After right join: {len(df_right)} rows")
print(f"\nKept all {len(stroke_df)} rows from stroke count table!")
print(f"Added IDS data for {df_right['components'].notna().sum()} characters")
print(f"Missing IDS data for {df_right['components'].isna().sum()} characters")
print("\nFirst few rows:")
print(df_right.head())
print("\nSome rows with missing IDS data:")
print(df_right[df_right['components'].isna()].head())


Original stroke count table: 89586 rows
After right join: 151936 rows

Kept all 89586 rows from stroke count table!
Added IDS data for 151660 characters
Missing IDS data for 276 characters

First few rows:
  code_point character components stroke_count
0     U+03B1         α          α            1
1     U+2113         ℓ          ℓ            1
2     U+2460         ①          ①            1
3     U+2461         ②          ②            2
4     U+2462         ③          ③            3

Some rows with missing IDS data:
       code_point   character components stroke_count
18            NaN           ⑰        NaN           17
19            NaN           ⑱        NaN           18
21            NaN           ⑳        NaN           20
151286        NaN  &CDP-854C;        NaN            8
151288        NaN  &CDP-8551;        NaN            7


## Comparing the Results

OK Marina, let's see what we learned. Why are the sizes different?

The key is understanding what each join type keeps:


In [30]:
# Compare the sizes
print("Summary of merge results:")
print(f"{'='*60}")
print(f"Original IDS table:           {len(ids_df):,} rows")
print(f"Original stroke count table:   {len(stroke_df):,} rows")
print(f"{'='*60}")
print(f"Inner join:  {len(df_inner):,} rows (only characters in BOTH tables)")
print(f"Left join:   {len(df_left):,} rows (all IDS + matching stroke counts)")
print(f"Right join:  {len(df_right):,} rows (all stroke counts + matching IDS)")
print(f"{'='*60}")
print(f"\nWhy is inner join smaller?")
print(f"  → Because {len(ids_df) - len(df_inner):,} characters in IDS table don't have stroke counts")
print(f"  → And {len(stroke_df) - len(df_inner):,} characters in stroke count table don't have IDS data")
print(f"\nWhy are left and right joins different sizes?")
print(f"  → Left join keeps all {len(ids_df):,} IDS characters (some without stroke counts)")
print(f"  → Right join keeps all {len(stroke_df):,} stroke count characters (some without IDS data)")
print(f"  → They're different because the two tables have different characters!")


Summary of merge results:
Original IDS table:           161,368 rows
Original stroke count table:   89,586 rows
Inner join:  151,660 rows (only characters in BOTH tables)
Left join:   161,368 rows (all IDS + matching stroke counts)
Right join:  151,936 rows (all stroke counts + matching IDS)

Why is inner join smaller?
  → Because 9,708 characters in IDS table don't have stroke counts
  → And -62,074 characters in stroke count table don't have IDS data

Why are left and right joins different sizes?
  → Left join keeps all 161,368 IDS characters (some without stroke counts)
  → Right join keeps all 89,586 stroke count characters (some without IDS data)
  → They're different because the two tables have different characters!


## To join me, you must join yourself

Perfect! Now you understand the difference. For most of your work, you'll probably want to use **left join** to keep all your IDS data and add stroke counts where available.

Of course, merging isn't just for combining different tables - you can also merge a table **onto itself**. 

Why the hell would you want to do that? Because it's metal. 🦄 WAAAAAAAAAAAAAAAAGH!

You don't believe me? Just fucking watch. 

OK, so you want to build something like [jisho.org](https://jisho.org/) for the character composer, right? Here's the problem, [jisho.org](https://jisho.org/) breaks characters into the 200 something radicals, but the IDS data divides things into semantic + phonetic, and the phonetic would need to be broken down further, right?  

First, let's define what we mean by "radicals".

- Go to [jisho.org](https://jisho.org/), and copy all the radicals from their little tray
- Paste into gedit
- Open find and replace
- Use regex to remove all numbers 
- Then, replace each word with that word surrounded by quotation marks and followed by a comma and a space.
- put brakets around it, and you have yourself a list.

In [31]:
# Make a list of standard radicals from Jisho.org
radical_list = [
    '一', '｜', '丶', 'ノ', '乙', '亅', '二', '亠', '人', '⺅', '𠆢', '儿', '入', 'ハ', '丷', '冂', '冖', '冫', 
    '几', '凵', '刀', '⺉', '力', '勹', '匕', '匚', '十', '卜', '卩', '厂', '厶', '又', 'マ', '九', 'ユ', '乃', 
    '𠂉', '⻌', '口', '囗', '土', '士', '夂', '夕', '大', '女', '子', '宀', '寸', '小', '⺌', '尢', '尸', '屮', 
    '山', '川', '巛', '工', '已', '巾', '干', '幺', '广', '廴', '廾', '弋', '弓', 'ヨ', '彑', '彡', '彳', '⺖', 
    '⺘', '⺡', '⺨', '⺾', '⻏', '⻖', '也', '亡', '及', '久', '⺹', '心', '戈', '戸', '手', '支', '攵', '文', '斗', 
    '斤', '方', '无', '日', '曰', '月', '木', '欠', '止', '歹', '殳', '比', '毛', '氏', '气', '水', '火', '⺣', 
    '爪', '父', '爻', '爿', '片', '牛', '犬', '⺭', '王', '元', '井', '勿', '尤', '五', '屯', '巴', '毋', '玄', 
    '瓦', '甘', '生', '用', '田', '疋', '疒', '癶', '白', '皮', '皿', '目', '矛', '矢', '石', '示', '禸', '禾', 
    '穴', '立', '⻂', '世', '巨', '冊', '母', '⺲', '牙', '瓜', '竹', '米', '糸', '缶', '羊', '羽', '而', '耒', 
    '耳', '聿', '肉', '自', '至', '臼', '舌', '舟', '艮', '色', '虍', '虫', '血', '行', '衣', '西', '臣', '見', 
    '角', '言', '谷', '豆', '豕', '豸', '貝', '赤', '走', '足', '身', '車', '辛', '辰', '酉', '釆', '里', '舛', 
    '麦', '金', '長', '門', '隶', '隹', '雨', '青', '非', '奄', '岡', '免', '斉', '面', '革', '韭', '音', '頁', 
    '風', '飛', '食', '首', '香', '品', '馬', '骨', '高', '髟', '鬥', '鬯', '鬲', '鬼', '竜', '韋', '魚', '鳥', 
    '鹵', '鹿', '麻', '亀', '啇', '黄', '黒', '黍', '黹', '無', '歯', '黽', '鼎', '鼓', '鼠', '鼻', '齊', '龠'
]

Shit, after testing, Daniel found that a good half of radicals went missing, so we need to add some things. Here is what Daniel came up with quickly:

In [32]:
# Let's add missing stuff
radical_list += [
    '阝', '釒', '亻', '艹', '忄', '扌', '氵', '犭', '礻', '罒', '辶', '牜', '八', '丨', '网', '丿', '刂', '𠂆',
    '黑', '𧾷', '龜', '龍', '攴'
]

### Level 1: More cleaning

First, we need to clean up the `components` column, distinguish between things that are and are not 'radicals', and offload contents to new columns. There are clever ways to do this, but we're in a hurry, so we'll do stuff the stupid way using the tools and knowledge that you've already acquired.

Two extra things:

- A DataFrame is basically a list of dictionaries. It can be turned into just that, and it can be remade from just that.
- We can iterrate through DataFrame rows with `.iterrows()` and convert rows to dictionaries with `.to_dict()`.


In [33]:
import re

# TODO There are duplicates in the character column, which should be manually cleaned
# For the moment, we'll just drop them
ids_df = ids_df.drop_duplicates(subset=['character'])

# Empty list to fill with dictionaries
ls = []

# Letters to use as column names
letters = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

# Iterate through the DataFrame
for idx, row in ids_df.iterrows():
    # Set letter index counter to 0
    letter_idx = 0
    # Convert the row to a dictionary
    d = row.to_dict()
    # Extract characters from the components column
    components = row['components'] 
    # Remove structure characters
    structure_chars = '⿰⿱⿲⿳⿴⿵⿶⿷⿸⿹⿺⿻'
    components = re.sub(f'[{structure_chars}]', '', components)
    # Add a radical string to the row dictionary
    d['radical_string'] = ''
    # Iterate through the characters
    for char in components:
        # If the character is in the list,
        if char in radical_list:
            # Then add it to the radical string
            d['radical_string'] += char
        # If the character ISN'T in the list,
        else:
            # Find the current letter from the letter index using slices
            letter = letters[letter_idx]
            # Add the character as the value of that key
            d[letter] = char
            # Increment the letter index counter
            letter_idx += 1
    # Append the dictionary to the list
    ls.append(d)

# Convert the list of dictionaries back to a DataFrame
ids_df = pd.DataFrame(ls)

# Fill NaN with empty string
ids_df['radical_string'] = ids_df['radical_string'].fillna('')

# Create all_components column from the original components column
# This removes structure characters but keeps all component characters (radicals + non-radicals)
def extract_all_components(components_str):
    """Extract all characters from components, removing only structure characters"""
    if pd.isna(components_str):
        return ''
    structure_chars = '⿰⿱⿲⿳⿴⿵⿶⿷⿸⿹⿺⿻'
    # Remove structure characters and arrows, keep everything else
    cleaned = re.sub(f'[{structure_chars}→←]', '', str(components_str))
    return cleaned

ids_df['all_components'] = ids_df['components'].apply(extract_all_components)

print("\nFirst few rows with extracted radicals:")
print(ids_df[['character', 'components', 'radical_string', 'all_components', 'a']].head(10))
temp = ids_df[ids_df['a'].isna()]
print(f"{len(temp)}/{len(ids_df)} rows are completely broken down into radicals")

# Back up
bu = ids_df.copy()


First few rows with extracted radicals:
  character components radical_string all_components    a
0         一          一              一              一  NaN
1         戌        ⿵戊一              一             戊一    戊
2         歲        ⿻步戌                            步戌    步
3         劌        ⿰歲刂              刂             歲刂    歲
4         噦        ⿰口歲              口             口歲    歲
5         奯        ⿱大歲              大             大歲    歲
6         濊        ⿰氵歲              氵             氵歲    歲
7         穢        ⿰禾歲              禾             禾歲    歲
8         翽        ⿰歲羽              羽             歲羽    歲
9         薉        ⿱艹歲              艹             艹歲    歲
12395/99016 rows are completely broken down into radicals


Fuck! That took Daniel's machine 7 seconds to calculate. That is an *eternity*. We'll be smarter next time so as not to waste all those precious seconds we could have used to think about important things, like email.

Anyways, now we have Level 1 radicals - the radicals that appear directly in each character's components. But what about characters that are made up of other characters, which are in turn made up of radicals? We need to go deeper!

### Level 2: Components of Components

Now we're going to merge the table onto itself. For each character in the components list that isn't a radical, we'll look up its components and extract radicals from those.

This is where self-joining gets interesting:

In [34]:
# Load backup
ids_df = bu.copy()

# We're going to make this a function, because we will reuse it later

def go_deeper(df, col):
    """
    Get radicals from next level (components of components)
    param df: DataFrame to work with
    param col: column to work with
    returns df: DataFrame to work with
    """
    # Reduce DataFrame to columns we need and make copy
    cc = df[[col, 'radical_string']].copy()
    # Fill empty radical_string with the character
    cc['radical_string'] = cc['radical_string'].fillna('')
    # Drop empty rows
    cc = cc[cc['radical_string'] != ''].copy()
    cc = cc.dropna(subset=col)
    cc = cc[cc[col] != ''].copy()
    # Rename columns
    cc = cc.rename(columns={col: 'character', 'radical_string': 'new_rads'})
    # Deduplicate again TODO this might be fucked
    cc = cc.drop_duplicates(subset=['character'])
    # Merge with original DataFrame
    df = pd.merge(df, cc, on='character', how='left')
    # Fill empty radical_string with the character
    df['new_rads'] = df['new_rads'].fillna(value='')
    # Split into those with and without new_rads
    df_with = df[df['new_rads'] != ''].copy()
    df_without = df[df['new_rads'] == ''].copy()
    # Add new_rads to radical_string
    df_with['radical_string'] = df_with['radical_string'] + df_with['new_rads']
    # For df_without, the component character doesn't have radicals in our lookup
    # We should NOT add the component character itself to radical_string (it's not a radical!)
    # So we keep radical_string unchanged for df_without
    # Concatenate the two DataFrames
    df = pd.concat([df_with, df_without])
    # Drop new_rads column
    df = df.drop(columns=['new_rads', col])
    # Drop duplicates
    df = df.drop_duplicates()
    # Fill empty radical_string with the character
    df['radical_string'] = df['radical_string'].fillna('')
    # Return the DataFrame
    return df

# Apply the function to column a
ids_df = go_deeper(ids_df, 'a')

print("First few rows:")
print(ids_df[['character', 'components', 'radical_string']].sample(10))
temp = ids_df[ids_df['b'].isna()]
print(f"{len(temp)}/{len(ids_df)} rows are completely broken down into radicals")

First few rows:
      character components radical_string
6584          𨜷        ⿰旁阝              阝
25270         㣸        ⿰彳㽔              彳
94264         𱌱        ⿰齿曳               
38971         𢙻  ⿱𦘒⿰心力[UT]             心力
76710         𬛧        ⿰𫇆子              子
81507         𭩽        ⿰木米            木米魚
91131         𰛮        ⿰氵卤              氵
70858         𪽺        ⿸痱気               
50632         𥢭        ⿰禾敝              禾
87964         𡭏      ⿱⑧⿰舌寸             舌寸
81563/99016 rows are completely broken down into radicals


### Level 3: Components of components of components

Excellent! Let's keep going then:


In [35]:
# Apply the function to column b
ids_df = go_deeper(ids_df, 'b')

print("First few rows:")
print(ids_df[['character', 'components', 'radical_string']].sample(10))
temp = ids_df[ids_df['c'].isna()]
print(f"{len(temp)}/{len(ids_df)} rows are completely broken down into radicals")

First few rows:
      character components radical_string
61195         𨄝        ⿱務足              足
89978         𰉎        ⿴囗爿             囗爿
12183         澆        ⿰氵堯              氵
34671         𠢬        ⿱無力             無力
71204         𪳇        ⿰木迦              木
80636         𭐲        ⿰夏番               
80275         𭊱        ⿰口跛              口
44587         𣙻        ⿰木菆              木
95081         𱙲    ⿰⿱韭井⿱丮女            韭井女
66987         𩩺        ⿰骨泉              骨
96380/99016 rows are completely broken down into radicals


One more time:

In [36]:
# Apply the function to column c
ids_df = go_deeper(ids_df, 'c')

print("First few rows:")
print(ids_df[['character', 'components', 'radical_string']].sample(10))
temp = ids_df[ids_df['d'].isna()]
print(f"{len(temp)}/{len(ids_df)} rows are completely broken down into radicals")

First few rows:
      character components radical_string
47716         𤒂        ⿰𤉭隹              隹
53935         𦀨        ⿰糹那               
6890          㫧        ⿱日米            日米木
16252         繡        ⿰糸肅              糸
4909          兊        ⿱公儿             儿亠
21607         熩        ⿰火扈              火
37819         𡝯        ⿰女隶             女隶
85594         𮤹        ⿰阝乇              阝
77447         𬚦        ⿰耳華              耳
98835         𮶨      ⿱艹⿵门禾             艹禾
97289/99016 rows are completely broken down into radicals


Screw it, that's probably good enough, let's test it out:

In [37]:
# Cut down to needed columns (keep all_components for searching)
ids_df = ids_df[['character', 'components', 'radical_string', 'all_components']].copy()

# Search in all_components (which contains all radicals and components, not just radicals)
# This allows us to find components like 舀 and 忄 that might not be radicals themselves
a = ids_df[
    ids_df['all_components'].str.contains('忄', na=False) &
    ids_df['all_components'].str.contains('舀', na=False)
]
print(f"Search for 忄 and 舀: {len(a)} results")
print(a)

b = ids_df[
    ids_df['all_components'].str.contains('車', na=False) &
    ids_df['all_components'].str.contains('鳥', na=False)
]
print(f"\nSearch for 車 and 鳥: {len(b)} results")
print(b)

# Save the final results for analysis in the next notebook
import os

folder_path = "../marina_tables"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Save the final merged table with all our work
ids_df.to_csv(os.path.join(folder_path, 'ids_with_radicals.csv'), 
              index=False, encoding='utf-8')
print(f"\n✅ Saved {len(ids_df)} rows to marina_tables/ids_with_radicals.csv")



Search for 忄 and 舀: 1 results
      character components radical_string all_components
16493         慆        ⿰忄舀              忄             忄舀

Search for 車 and 鳥: 1 results
      character components radical_string all_components
27368         鿂        ⿰車鳥             車鳥             車鳥

✅ Saved 99016 rows to marina_tables/ids_with_radicals.csv


Awesome! Now you have a column that lists all the radical components for each character, going 2-3 levels deep. 

## What We Learned

1. **Inner Join**: Only keeps rows where the key exists in BOTH tables
2. **Left Join**: Keeps all rows from the LEFT table, adds matching data from RIGHT
3. **Right Join**: Keeps all rows from the RIGHT table, adds matching data from LEFT
4. **Self-Join**: Merge a table onto itself to find relationships within the same data

## Summary

| Merge Type | `how=` | Keeps Rows From | Use When |
|------------|--------|------------------|----------|
| Inner | `'inner'` | Both DataFrames | You only want matching records |
| Left | `'left'` | Left DataFrame | Keep all from left, add matching from right |
| Right | `'right'` | Right DataFrame | Keep all from right, add matching from left |

## Key Parameters

- `on='column'` - Merge on a column with the same name in both DataFrames
- `left_on='col1', right_on='col2'` - Merge on columns with different names
- `how='inner'` - Type of join (inner, left, right, outer)

## What's Next?

In the next notebook, we'll learn how to:
- Transform and manipulate data
- Create new columns
- Build lookup tables for dictionaries and character composers
